In [34]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import uqer
from PyFin.api import *
from uqer import DataAPI as api
from QuantLib import *

plt.style.use('fivethirtyeight')
uqer.Client(token='f1b9bea1d0b4e489c5ab9b69c3e2326a1bee6057af858067dbd1546453f428b2')

16937@wmcloud.com 账号登录成功


In [35]:
start_date = '2017-04-01'
end_date = '2017-08-01'

dates = makeSchedule(start_date, end_date, '1m', 'china.sse', )

mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
contract = 'ru1709'

In [36]:
def get_contracts(contract, start_date, end_date):
    data = api.MktFutdGet(ticker=contract, beginDate=start_date, endDate=end_date, field='tradeDate,closePrice').set_index('tradeDate')
    data['closePrice'] = data['closePrice'].astype(float)
    data['ret'] = data.closePrice.diff() / data.closePrice.shift(1)
    data['vol'] = data['ret'].rolling(window=22).std() * np.sqrt(249)
    return data

In [37]:
mkt_data = get_contracts(contract, mkt_start_date, end_date)

In [38]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.

# Dynamical Hedging
------------

In [39]:
def hedging_contract(start_date, maturity_date, mkt_data, premium=None):
    spot_price = mkt_data.loc[start_date.ISO()]['closePrice']
    lower_strike = spot_price
    lower_bound = lower_strike - lower_protect_area
    upper_strike = lower_strike + strangle_width
    upper_bound = upper_strike + upper_protect_area
    
    day_count = Actual365Fixed()
    calendar = China(China.SSE)
    
    calculation_date = start_date
    Settings.instance().evaluationDate = calculation_date

    spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
    vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

    flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
    dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
    flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

    bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
    
    put_payoff1 = PlainVanillaPayoff(Option.Put, lower_strike)
    put_payoff2 = PlainVanillaPayoff(Option.Put, lower_bound)
    call_payoff1 = PlainVanillaPayoff(Option.Call, upper_strike)
    call_payoff2 = PlainVanillaPayoff(Option.Call, upper_bound)

    exercise = EuropeanExercise(maturity_date)
    put_option1 = VanillaOption(put_payoff1, exercise)
    put_option2 = VanillaOption(put_payoff2, exercise)
    call_option1 = VanillaOption(call_payoff1, exercise)
    call_option2 = VanillaOption(call_payoff2, exercise)

    engine = AnalyticEuropeanEngine(bsm_process)
    put_option1.setPricingEngine(engine)
    put_option2.setPricingEngine(engine)
    call_option1.setPricingEngine(engine)
    call_option2.setPricingEngine(engine)
    
    if not premium:
        premium = put_option1.NPV() - put_option2.NPV() + call_option1.NPV() - call_option2.NPV()
    
    price_series = mkt_data.loc[start_date.ISO(): maturity_date.ISO()]
    previous_delta = 0.
    previous_price = 0.
    trading_pnl = 0.
    for p in price_series.iterrows():
        date = Date.from_date(dt.datetime.strptime(p[0], '%Y-%m-%d'))
        price = p[1]['closePrice']
        vol = p[1]['vol']
        Settings.instance().evaluationDate = date
        spot_handle.linkTo(SimpleQuote(price))
        vol_handle.linkTo(SimpleQuote(vol))
        bsm_delta = put_option1.delta() - put_option2.delta() + call_option1.delta() - call_option2.delta()
        trading_pnl += (price - previous_price) * previous_delta
        previous_price = price
        previous_delta = bsm_delta
        
    final_price = mkt_data.loc[maturity_date.ISO()]['closePrice']
    final_payoff = put_payoff1(final_price) - put_payoff2(final_price) + call_payoff1(final_price) - call_payoff2(final_price)
    
    pnl_final = -final_payoff + premium + trading_pnl
    
    return spot_price, final_price, vol, final_payoff, premium, trading_pnl, pnl_final

In [40]:
maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[3], '-1b'))
start_date = Date.from_date(dates[2])

hedging_contract(start_date, maturity_date, mkt_data, 646.)

(12480.0,
 13335.0,
 0.26391057260945405,
 455.0,
 646.0,
 -100.0891763893071,
 90.910823610692901)

# Historical Simulation
-----------------

In [41]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.

In [42]:
contract_definition = {
    'ru1309': {
        'start_date': '2013-04-01',
        'end_date': '2013-08-01'
    },
    'ru1401': {
        'start_date': '2013-08-01',
        'end_date': '2013-12-01'
    },
    'ru1409': {
        'start_date': '2014-04-01',
        'end_date': '2014-08-01'
    },
     'ru1501': {
        'start_date': '2014-08-01',
        'end_date': '2014-12-01'
    },
    'ru1509': {
        'start_date': '2015-04-01',
        'end_date': '2015-08-01'
    },
    'ru1601': {
        'start_date': '2015-08-01',
        'end_date': '2015-12-01'
    },
    'ru1609': {
        'start_date': '2016-04-01',
        'end_date': '2016-08-01'
    },
    'ru1701': {
        'start_date': '2016-08-01',
        'end_date': '2016-12-01'
    },
    'ru1709': {
        'start_date': '2017-04-01',
        'end_date': '2017-08-01'
    },
    'ru1801': {
        'start_date': '2017-08-01',
        'end_date': '2017-12-01'
    },
}

In [51]:
cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益']
df = pd.DataFrame(columns=cols)

for contract in contract_definition:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '1m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts(contract, mkt_start_date, end_date)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        res = hedging_contract(start_date, maturity_date, mkt_data, 723.)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df = df.append(dict(zip(cols, (contract, py_dt) + res)), ignore_index=True)

In [52]:
df['不对冲损益'] = -df['赔付'] + df['期权费']

In [53]:
df

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,不对冲损益
0,ru1309,2013-04-26,21460.0,19080.0,0.323892,1000.0,723.0,295.470782,18.470782,-277.0
1,ru1309,2013-05-31,19120.0,18740.0,0.312908,380.0,723.0,-69.150455,273.849545,343.0
2,ru1309,2013-06-28,18955.0,17005.0,0.273979,1000.0,723.0,263.903523,-13.096477,-277.0
3,ru1309,2013-07-31,17275.0,16865.0,0.295579,410.0,723.0,-59.450781,253.549219,313.0
4,ru1401,2013-08-30,18110.0,20000.0,0.246227,800.0,723.0,-13.336270,-90.336270,-77.0
5,ru1401,2013-09-30,21055.0,20080.0,0.263100,975.0,723.0,73.112285,-178.887715,-252.0
6,ru1401,2013-10-31,20585.0,19335.0,0.234116,1000.0,723.0,117.325512,-159.674488,-277.0
7,ru1401,2013-11-29,19385.0,19045.0,0.142093,340.0,723.0,-105.741697,277.258303,383.0
8,ru1409,2014-04-30,15785.0,14340.0,0.238007,1000.0,723.0,294.105265,17.105265,-277.0
9,ru1409,2014-05-30,14365.0,14125.0,0.270710,240.0,723.0,-139.384254,343.615746,483.0


In [55]:
df.groupby(df['到期日'].dt.year).mean()

,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,不对冲损益
到期日,,,,,,,,
2013,19493.125,18768.750,0.261487,738.125,723.0,62.766612,47.641612,-15.125
2014,14331.250,13788.125,0.229827,628.125,723.0,138.901017,233.776017,94.875
2015,12715.000,12336.250,0.264896,566.875,723.0,87.825971,243.950971,156.125
2016,12380.625,12777.500,0.317360,626.250,723.0,34.778044,131.528044,96.750
2017,14629.375,13744.375,0.333330,673.750,723.0,54.378138,103.628138,49.250
